In [1]:
from hops import hdfs
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import LongType
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1559755371379_0026,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

In [2]:
    dfCold = spark \
        .read \
        .format("parquet") \
        .load(hdfs.project_path() + "Resources/iot-benchmarks/data/topic-lwm2m-3303-temperature") \
        .filter(F.col('endpointClientName').contains('node-cold-20'))

    dfCold.cache().count()

149

In [3]:
    dfWarm = spark \
        .read \
        .format("parquet") \
        .load(hdfs.project_path() + "Resources/iot-benchmarks/data/topic-lwm2m-3303-temperature") \
        .filter(F.col('endpointClientName').contains('node-warm-20'))

    dfWarm.cache().count()

151

In [4]:
timeFmt = "yyyy-MM-dd HH:mm:ss.SSS"
sensDfCold = dfCold \
    .select(F.to_timestamp(F.col('measurement.timestamp')/1000).alias('measurementTmp'), \
            'kafkaTimestamp', \
            (F.col('measurement.timestamp')).alias('measurementTmpLong'), \
            'endpointClientName') \
    .withColumn('kafkaTmpLong', F.substring_index('kafkaTimestamp', '.', -1).cast('float')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 10, F.col('kafkaTmpLong')*100).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 100, F.col('kafkaTmpLong')*10).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', (F.unix_timestamp('kafkaTimestamp', format=timeFmt)*1000 + F.col('kafkaTmpLong'))) \
    .withColumn("diffMilliseconds", (F.col('kafkaTmpLong') - F.col('measurementTmpLong')))
    

In [5]:
sensDfWarm = dfWarm \
    .select(F.to_timestamp(F.col('measurement.timestamp')/1000).alias('measurementTmp'), \
            'kafkaTimestamp', \
            (F.col('measurement.timestamp')).alias('measurementTmpLong'), \
            'endpointClientName') \
    .withColumn('kafkaTmpLong', F.substring_index('kafkaTimestamp', '.', -1).cast('float')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 10, F.col('kafkaTmpLong')*100).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 100, F.col('kafkaTmpLong')*10).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', (F.unix_timestamp('kafkaTimestamp', format=timeFmt)*1000 + F.col('kafkaTmpLong'))) \
    .withColumn("diffMilliseconds", (F.col('kafkaTmpLong') - F.col('measurementTmpLong')))
    

In [6]:
sensDfCold.agg(F.avg(F.col('diffMilliseconds'))).show()

+---------------------+
|avg(diffMilliseconds)|
+---------------------+
|    97.81879194630872|
+---------------------+

In [7]:
sensDfWarm.agg(F.avg(F.col('diffMilliseconds'))).show()

+---------------------+
|avg(diffMilliseconds)|
+---------------------+
|    63.99337748344371|
+---------------------+

In [8]:
%%spark -o sensDfCold

In [9]:
%%local
sensDfCold

/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()

In [10]:
%%spark -o sensDfWarm

In [11]:
%%local
sensDfWarm

/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()